# Import a model

|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating neurons and dimensions<h1>|
|<h2>Lecture:</h2>|<h1><b>Clarification of final hidden_states output<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

n_layers = model.config.n_layer
model.eval()

# Implant a hook

In [ ]:
blockOutput = {'data':4.}

def hook(module, input, output):
  blockOutput['data'] = output[0]

model.transformer.h[n_layers-1].register_forward_hook(hook)

# Forward pass

In [ ]:
with torch.no_grad():
  out = model(tokenizer.encode('hello',return_tensors='pt'),
              output_hidden_states=True
              )

In [ ]:
print('Hooked data are of size',blockOutput['data'].shape)
print('Final hidden_state is of size',out.hidden_states[-1].shape)

# Compare hidden_state to pre- and post-LN

In [ ]:
# pre-LN
blockOutput['data'][0,-1,:] - out.hidden_states[-1][0,-1,:]

In [ ]:
# post-LN
postLayerNorm = model.transformer.ln_f( blockOutput['data'] )
postLayerNorm[0,-1,:] - out.hidden_states[-1][0,-1,:]